In [1]:
import cv2 as cv
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np
from sklearn.cluster import KMeans
import os, shutil, glob, os.path
import tensorflow as tf
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_curve, auc
from sklearn.cluster import Birch
import seaborn as sns
sns.set()
#pip install pillow 
#pip install opencv-python==3.4.2.16
#pip install opencv-contrib-python==3.4.2.16 --user

Using TensorFlow backend.


In [2]:
COVIDImages = "COVIDImages"
OtherImages = "OtherImages"
filelists = []

In [3]:
imdir1 = COVIDImages
filelist1 = glob.glob(os.path.join(imdir1, '*'))
filelist1.sort()
filelists.append(filelist1)
imdir2 = OtherImages
filelist2 = glob.glob(os.path.join(imdir2, '*'))
filelist2.sort()
filelists.append(filelist2)

In [4]:
featurelist = []
descriptors = []
actual_labels = []

In [5]:
#SIFT is only available in pip install --user opencv-contrib-python==3.4.2.17
WIDTH = 250
HEIGHT = 250
for filelist in filelists:
    for i, imagepath in enumerate(filelist):
        print("    Status: %s / %s" %(i, len(filelist)), end="\r")
    #     img = image.load_img(imagepath, target_size=(224, 224))
    #     img_data = image.img_to_array(img)
    #     img_data = np.expand_dims(img_data, axis=0)#Insert a new axis that will appear at the axis position in the expanded array shape.
    #     img_data = preprocess_input(img_data)
    #     features = np.array(model.predict(img_data))
    #     featurelist.append(features.flatten())
        a = 0 if any("COVID" in s for s in filelist) else 1
        actual_labels.append(a)
        img = cv.imread(imagepath)
        d = cv.resize(img, (WIDTH,HEIGHT), interpolation=cv.INTER_CUBIC)
        blur = cv.GaussianBlur(img, (5, 5), 0)
        gray= cv.cvtColor(blur,cv.COLOR_BGR2GRAY)
        sift = cv.xfeatures2d.SIFT_create()
        kp = sift.detect(gray,None)
        img=cv.drawKeypoints(gray,kp,outImage = None)
    #     cv.imwrite('sift_keypoints.jpg',img)
        kp, des = sift.detectAndCompute(gray,None)
        feature = np.array(des)
        descriptors.append(feature.flatten())

In [6]:
# for i in range(len(descriptors)):
#     descriptors[i] = descriptors[i].flatten()
descriptors

[array([18.,  0.,  0., ...,  0.,  0.,  7.], dtype=float32),
 array([0., 1., 8., ..., 4., 0., 3.], dtype=float32),
 array([ 6.,  0.,  0., ..., 13., 16., 14.], dtype=float32),
 array([ 0.,  0.,  0., ..., 20.,  1.,  1.], dtype=float32),
 array([0., 0., 7., ..., 1., 0., 0.], dtype=float32),
 array([ 0.,  0.,  0., ...,  3.,  9., 27.], dtype=float32),
 array([ 0., 37., 49., ..., 39.,  2.,  1.], dtype=float32),
 array([145.,   8.,   0., ...,   0.,   0.,   0.], dtype=float32),
 array([10., 29.,  3., ...,  0.,  0.,  0.], dtype=float32),
 array([3., 0., 0., ..., 0., 0., 4.], dtype=float32),
 array([ 1., 45., 75., ...,  0.,  0.,  8.], dtype=float32),
 array([20.,  7.,  0., ...,  0.,  0., 10.], dtype=float32),
 array([0., 0., 0., ..., 0., 0., 1.], dtype=float32),
 array([44.,  5.,  1., ..., 46., 11., 32.], dtype=float32),
 array([92., 23.,  0., ...,  4., 96., 63.], dtype=float32),
 array([ 3.,  9., 52., ...,  0.,  0.,  0.], dtype=float32),
 array([124., 136.,  27., ...,   0.,   0.,   1.], dtype=fl

In [7]:
# featurelist[0:5]
# len(descriptors[5])
# store = [len(x) for x in descriptors]
# store
max_len = np.max([len(a) for a in descriptors])
descriptor = np.asarray([np.pad(a, (0, max_len - len(a)), 'constant', constant_values=0.) for a in descriptors])

In [9]:
# featurelist[0][7]
descriptors[0:5]

[array([18.,  0.,  0., ...,  0.,  0.,  7.], dtype=float32),
 array([0., 1., 8., ..., 4., 0., 3.], dtype=float32),
 array([ 6.,  0.,  0., ..., 13., 16., 14.], dtype=float32),
 array([ 0.,  0.,  0., ..., 20.,  1.,  1.], dtype=float32),
 array([0., 0., 7., ..., 1., 0., 0.], dtype=float32)]

In [10]:
kmeans = KMeans(n_clusters=2, random_state=0).fit(np.array(descriptor))

In [11]:
kmeans.labels_

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0,

In [12]:
from sklearn.metrics import accuracy_score 
pred = kmeans.labels_
score = round(accuracy_score(pred, actual_labels), 4)
print('Accuracy scored using k-means clustering: ', score)

Accuracy scored using k-means clustering:  0.7225


In [13]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram

In [14]:
model = AgglomerativeClustering(n_clusters=2,linkage="ward")
model.fit(np.array(descriptor))
model.labels_
x = model.labels_
score = round(accuracy_score(x, actual_labels), 4)
print('Accuracy scored using Agglomerative clustering: ', score)

Accuracy scored using Agglomerative clustering:  0.2632


In [15]:
model1 = AgglomerativeClustering(n_clusters=2, compute_full_tree=True,
    affinity='euclidean', linkage='complete')
model1.fit(np.array(descriptor))
model1.labels_
x1 = model1.labels_
score = round(accuracy_score(x1, actual_labels), 4)
print('Accuracy scored using complete linkage Agglomerative clustering: ', score)

Accuracy scored using complete linkage Agglomerative clustering:  0.4976


In [16]:
model2 = AgglomerativeClustering(n_clusters=2, compute_full_tree=True,
    affinity='euclidean', linkage='single')
model2.fit(np.array(descriptor))
model2.labels_
x2 = model2.labels_
score = round(accuracy_score(x2, actual_labels), 4)
print('Accuracy scored using complete linkage Agglomerative clustering: ', score)

Accuracy scored using complete linkage Agglomerative clustering:  0.4976


In [17]:
bclust=Birch(branching_factor=100, threshold=.3,n_clusters=2).fit(np.array(descriptor))
print(bclust)
labels_birch = bclust.predict(np.array(descriptor))
labels_birch

Birch(branching_factor=100, compute_labels=True, copy=True, n_clusters=2,
      threshold=0.3)


array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,

In [18]:
for i in range(len(labels_birch)):
    if(labels_birch[i]==0):
        labels_birch[i]=1
    else:
        labels_birch[i]=0

In [19]:
score = round(accuracy_score(labels_birch, actual_labels), 4)
print('Accuracy scored using birch clustering: ', score)

Accuracy scored using birch clustering:  0.7368


In [20]:
conf_mat_complete = confusion_matrix(actual_labels, labels_birch)
print(conf_mat_complete)
cr_complete=classification_report(actual_labels, labels_birch)
print(cr_complete)

[[182  27]
 [ 83 126]]
              precision    recall  f1-score   support

           0       0.69      0.87      0.77       209
           1       0.82      0.60      0.70       209

    accuracy                           0.74       418
   macro avg       0.76      0.74      0.73       418
weighted avg       0.76      0.74      0.73       418

